<a href="https://colab.research.google.com/github/Degananda264/Natural-Language-Processing-/blob/master/Document_Clustering_using_statistical_%26_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install textsearch
!pip install contractions
!pip install textsearch
import nltk
nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 245kB 3.7MB/s 
     |████████████████████████████████| 317kB 16.3MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81703 sha256=f2d704be68a5be70753a2042e83c669b78c10ea19cb886db32ad7dceaa83fa06
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv("movies.csv")

In [5]:
df.shape

(4803, 20)

In [6]:
df.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [7]:
df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [8]:
df = df[['title', 'tagline', 'overview', 'popularity']]

In [9]:
df.head()

,title,tagline,overview,popularity
0,Avatar,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",150.437577
1,Pirates of the Caribbean: At World's End,"At the end of the world, the adventure begins.","Captain Barbossa, long believed to be dead, ha...",139.082615
2,Spectre,A Plan No One Escapes,A cryptic message from Bond’s past sends him o...,107.376788
3,The Dark Knight Rises,The Legend Ends,Following the death of District Attorney Harve...,112.312950
4,John Carter,"Lost in our world, found in another.","John Carter is a war-weary, former military ca...",43.926995


In [10]:
df.tagline.fillna('', inplace=True)
df['description'] = df['tagline'].map(str) + ' ' + df['overview']
df.dropna(inplace=True)

In [11]:
df.head()

,title,tagline,overview,popularity,description
0,Avatar,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",150.437577,Enter the World of Pandora. In the 22nd centur...
1,Pirates of the Caribbean: At World's End,"At the end of the world, the adventure begins.","Captain Barbossa, long believed to be dead, ha...",139.082615,"At the end of the world, the adventure begins...."
2,Spectre,A Plan No One Escapes,A cryptic message from Bond’s past sends him o...,107.376788,A Plan No One Escapes A cryptic message from B...
3,The Dark Knight Rises,The Legend Ends,Following the death of District Attorney Harve...,112.312950,The Legend Ends Following the death of Distric...
4,John Carter,"Lost in our world, found in another.","John Carter is a war-weary, former military ca...",43.926995,"Lost in our world, found in another. John Cart..."


In [12]:
df.isnull().sum()

title          0
tagline        0
overview       0
popularity     0
description    0
dtype: int64

**Clustering Similar Movies**

In [13]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['description']))
len(norm_corpus)

4800

In [14]:
norm_corpus

array(['enter world pandora 22nd century paraplegic marine dispatched moon pandora unique mission becomes torn following orders protecting alien civilization',
       'end world adventure begins captain barbossa long believed dead come back life headed edge earth turner elizabeth swann nothing quite seems',
       'plan one escapes cryptic message bonds past sends trail uncover sinister organization battles political forces keep secret service alive bond peels back layers deceit reveal terrible truth behind spectre',
       ...,
       'signed sealed delivered introduces dedicated quartet civil servants dead letter office us postal system transform elite team lostmail detectives determination deliver seemingly undeliverable takes post office unpredictable world letters packages past save lives solve crimes reunite old loves change futures arriving late always miraculously time',
       'new yorker shanghai ambitious new york attorney sam sent shanghai assignment immediately stumbles le

**Extract TF-IDF Features**

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

stop_words = stop_words + ['one', 'two', 'three', 'four', 'five', 'get']
cv = CountVectorizer(ngram_range=(1, 2), min_df=0.001, max_df=0.99, stop_words=stop_words)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix.shape

(4800, 6101)

**K-Means Clustering**

In [16]:
#using Elbow method to find K- value
from sklearn.cluster import KMeans

NUM_CLUSTERS = 6
km = KMeans(n_clusters=NUM_CLUSTERS, max_iter=10000, n_init=100, random_state=42).fit(cv_matrix)
km

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=10000,
       n_clusters=6, n_init=100, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [17]:
from collections import Counter

Counter(km.labels_)

Counter({0: 3313, 1: 454, 2: 241, 3: 626, 4: 159, 5: 7})

In [18]:
df['kmeans_cluster'] = km.labels_

In [19]:

df.head(5)

,title,tagline,overview,popularity,description,kmeans_cluster
0,Avatar,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",150.437577,Enter the World of Pandora. In the 22nd centur...,0
1,Pirates of the Caribbean: At World's End,"At the end of the world, the adventure begins.","Captain Barbossa, long believed to be dead, ha...",139.082615,"At the end of the world, the adventure begins....",3
2,Spectre,A Plan No One Escapes,A cryptic message from Bond’s past sends him o...,107.376788,A Plan No One Escapes A cryptic message from B...,0
3,The Dark Knight Rises,The Legend Ends,Following the death of District Attorney Harve...,112.312950,The Legend Ends Following the death of Distric...,2
4,John Carter,"Lost in our world, found in another.","John Carter is a war-weary, former military ca...",43.926995,"Lost in our world, found in another. John Cart...",0


In [20]:
movie_clusters = (df[['title', 'kmeans_cluster', 'popularity']]
                  .sort_values(by=['kmeans_cluster', 'popularity'], 
                               ascending=False)
                  .groupby('kmeans_cluster').head(20))
movie_clusters = movie_clusters.copy(deep=True)

In [21]:
movie_clusters

,title,kmeans_cluster,popularity
1857,Rugrats Go Wild,5,8.255708
4731,Indie Game: The Movie,5,7.676677
4110,Inside Deep Throat,5,3.980072
2873,Girl 6,5,2.993939
4154,Slacker Uprising,5,1.619635
...,...,...,...
12,Pirates of the Caribbean: Dead Man's Chest,0,145.847379
16,The Avengers,0,144.448633
693,Gone Girl,0,143.041543
456,Fury,0,139.575085


In [22]:
feature_names = cv.get_feature_names()
topn_features = 30
ordered_centroids = km.cluster_centers_.argsort()[:, ::-1]

# get key features for each cluster
# get movies belonging to each cluster
for cluster_num in range(NUM_CLUSTERS):
    key_features = [feature_names[index] 
                        for index in ordered_centroids[cluster_num, :topn_features]]
    movies = movie_clusters[movie_clusters['kmeans_cluster'] == cluster_num]['title'].values.tolist()
    print('CLUSTER #'+str(cluster_num+1))
    print('Key Features:', key_features)
    print('Popular Movies:', movies)
    print('-'*80)

CLUSTER #1
Key Features: ['world', 'young', 'man', 'family', 'find', 'must', 'new', 'time', 'back', 'friends', 'years', 'way', 'story', 'war', 'help', 'home', 'town', 'lives', 'hes', 'finds', 'take', 'film', 'father', 'become', 'school', 'evil', 'becomes', 'group', 'save', 'team']
Popular Movies: ['Minions', 'Interstellar', 'Guardians of the Galaxy', 'Jurassic World', 'Dawn of the Planet of the Apes', 'The Hunger Games: Mockingjay - Part 1', 'Big Hero 6', 'Terminator Genisys', 'Captain America: Civil War', 'Whiplash', 'The Dark Knight', 'The Martian', 'Batman v Superman: Dawn of Justice', 'Avatar', 'Fight Club', "Pirates of the Caribbean: Dead Man's Chest", 'The Avengers', 'Gone Girl', 'Fury', 'X-Men: Apocalypse']
--------------------------------------------------------------------------------
CLUSTER #2
Key Features: ['love', 'life', 'man', 'find', 'story', 'young', 'falls', 'world', 'woman', 'finds', 'family', 'new', 'falls love', 'father', 'fall', 'true', 'girl', 'must', 'time', 'ne

**Clustering with Embeddings**

In [23]:
from gensim.models import FastText
tokenized_docs = [doc.split() for doc in norm_corpus]
ft_model = FastText(tokenized_docs, size=300, window=40, min_count=2, workers=4, sg=1, iter=100)

In [26]:
print(norm_corpus)

['enter world pandora 22nd century paraplegic marine dispatched moon pandora unique mission becomes torn following orders protecting alien civilization'
 'end world adventure begins captain barbossa long believed dead come back life headed edge earth turner elizabeth swann nothing quite seems'
 'plan one escapes cryptic message bonds past sends trail uncover sinister organization battles political forces keep secret service alive bond peels back layers deceit reveal terrible truth behind spectre'
 ...
 'signed sealed delivered introduces dedicated quartet civil servants dead letter office us postal system transform elite team lostmail detectives determination deliver seemingly undeliverable takes post office unpredictable world letters packages past save lives solve crimes reunite old loves change futures arriving late always miraculously time'
 'new yorker shanghai ambitious new york attorney sam sent shanghai assignment immediately stumbles legal mess could end career help beautiful 

In [25]:
print(tokenized_docs)

[['enter', 'world', 'pandora', '22nd', 'century', 'paraplegic', 'marine', 'dispatched', 'moon', 'pandora', 'unique', 'mission', 'becomes', 'torn', 'following', 'orders', 'protecting', 'alien', 'civilization'], ['end', 'world', 'adventure', 'begins', 'captain', 'barbossa', 'long', 'believed', 'dead', 'come', 'back', 'life', 'headed', 'edge', 'earth', 'turner', 'elizabeth', 'swann', 'nothing', 'quite', 'seems'], ['plan', 'one', 'escapes', 'cryptic', 'message', 'bonds', 'past', 'sends', 'trail', 'uncover', 'sinister', 'organization', 'battles', 'political', 'forces', 'keep', 'secret', 'service', 'alive', 'bond', 'peels', 'back', 'layers', 'deceit', 'reveal', 'terrible', 'truth', 'behind', 'spectre'], ['legend', 'ends', 'following', 'death', 'district', 'attorney', 'harvey', 'dent', 'batman', 'assumes', 'responsibility', 'dents', 'crimes', 'protect', 'late', 'attorneys', 'reputation', 'subsequently', 'hunted', 'gotham', 'city', 'police', 'department', 'eight', 'years', 'later', 'batman', '

In [27]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [28]:
doc_vecs_ft = averaged_word2vec_vectorizer(tokenized_docs, ft_model, 300)
doc_vecs_ft.shape

(4800, 300)

In [30]:
doc_vecs_ft[:5]

array([[ 0.28500968, -0.09547904, -0.0331859 , ..., -0.35576504,
        -0.16643492, -0.13060903],
       [ 0.08119409, -0.00396194,  0.01325562, ..., -0.23424639,
        -0.10761693,  0.09150613],
       [ 0.36128457, -0.16461676, -0.02185896, ..., -0.10422557,
        -0.13434523,  0.10566406],
       [ 0.16580356,  0.04283393,  0.0653375 , ..., -0.06087746,
        -0.08846564, -0.00420777],
       [ 0.2840548 , -0.04578021,  0.14306134, ..., -0.24741926,
        -0.04518811,  0.13221185]])

In [31]:
NUM_CLUSTERS = 6
km = KMeans(n_clusters=NUM_CLUSTERS, max_iter=10000, n_init=100, random_state=42).fit(doc_vecs_ft)

In [32]:
Counter(km.labels_)

Counter({0: 1019, 1: 854, 2: 1165, 3: 610, 4: 594, 5: 558})

In [33]:
df['kmeans_cluster'] = km.labels_
movie_clusters = (df[['title', 'kmeans_cluster', 'popularity']]
                  .sort_values(by=['kmeans_cluster', 'popularity'], 
                               ascending=False)
                  .groupby('kmeans_cluster').head(30))
movie_clusters = movie_clusters.copy(deep=True)

In [34]:

for cluster_num in range(NUM_CLUSTERS):
    movies = movie_clusters[movie_clusters['kmeans_cluster'] == cluster_num]['title'].values.tolist()
    print('CLUSTER #'+str(cluster_num+1))
    print('Popular Movies:', movies)
    print('-'*80)

CLUSTER #1
Popular Movies: ['Fight Club', 'Gone Girl', 'Inside Out', 'Despicable Me', "Harry Potter and the Philosopher's Stone", 'Bruce Almighty', 'The Twilight Saga: Eclipse', 'Titanic', 'Fifty Shades of Grey', 'Psycho', 'The Amazing Spider-Man', 'Birdman', 'Monsters University', 'Finding Nemo', 'The Devil Wears Prada', 'Cars', 'Spider-Man', 'The Hangover', 'Horrible Bosses 2', 'Home Alone 2: Lost in New York', 'American Beauty', 'Back to the Future', 'Amélie', 'Toy Story', 'Toy Story 2', 'Rocky', 'Pitch Perfect 2', 'Ted 2', 'The Wizard of Oz', 'Sausage Party']
--------------------------------------------------------------------------------
CLUSTER #2
Popular Movies: ['Minions', 'The Dark Knight', 'Inception', 'The Godfather', 'Pixels', 'Despicable Me 2', 'The Shawshank Redemption', "One Flew Over the Cuckoo's Nest", 'Mission: Impossible - Rogue Nation', 'The Dark Knight Rises', 'Quantum of Solace', 'The Godfather: Part II', 'Furious 7', "We're the Millers", 'I, Robot', 'The Wolf of 